In [1]:
import numpy as np
import os
import random

def buffer(x: np.ndarray):
    return x

def inverter(x: np.ndarray):
    return np.logical_not(x)

def AND(x: np.ndarray):
    return np.logical_and(x[0], x[1])
    
def NAND(x: np.ndarray):
    return np.logical_not(np.logical_and(x[0], x[1]))

def OR(x: np.ndarray):
    return np.logical_or(x[0], x[1])
    
def NOR(x: np.ndarray):
    return np.logical_not(np.logical_or(x[0], x[1]))

basics = {
    'BUF': buffer,
    'INV': inverter,
    'AND': AND,
    'NAND': NAND,
    'OR': OR,
    'NOR': NOR,
    # 'INPUT' = INPUT,
    # 'OUTPUT' = OUTPUT
}

class Circuit:
    
    # initialize the circuit
    # :param file: a file detailing the circuit gates, nodes, and I/O
    def __init__(self, file):
        self.parse(file)
        self.file = file
    
    # parse the input file
    def parse(self, file):
        # open file
        with open(file) as f:
            circuit = f.read()
            
        # double spaces are annoying
        circuit.replace('  ', ' ')
        
        # create an empty dictionary
        self.circuit = {}
        
        # iterate over each line in the file (each gate or input/output)
        for i, element in enumerate(circuit.split('\n')):
            # ignore empty strings/lines
            if element == '':
                continue
            # split the line into each of its components
            info = element.split(' ')
            
            # if the first element is input or output, uses a slightly different format
            # {INPUT: [1, 2, 3...]}
            if info[0] in ['INPUT', 'OUTPUT']:
                self.circuit.update({info[0]: list(map(int, info[2:]))})
            # else use this format
            # {GATE_#: {IN: [1, 2], OUT: [3]}}
            else:
                self.circuit.update({info[0]+f'_{i}': {'IN': list(map(int, info[1:-1])), 'OUT': int(info[-1])}})
        
        # find the largest number in the file
        self.length = 0
        for line in circuit.split('\n'):
            for num in line.split(' '):
                try:
                    num = int(num)
                    if num > self.length:
                        self.length = num
                except:
                    continue
        # add two for direct indexing
        # i.e. index 0 and -1 are ignored, so index 1 -> circuit[1]
        self.length+=2
            
    # simulate the circuit until steady state is reached
    def simulate(self, input_vector):
        # initialize the circuit with all 0s
        self.start = np.asarray([0]*self.length)
        self.next = np.copy(self.start)
        
        print('Input: ' + input_vector)
        # add the input values
        for i, index in zip(input_vector[:], self.circuit['INPUT']):
            self.next[index] = int(i)
        # print(self.next[1:])
        
        print(self.start)
        print(len(self.start))
        # while not in a steady state, continue looping over the gates in the circuit
        while not np.array_equal(self.start, self.next):
            self.start = np.copy(self.next)
            for element in self.circuit.keys():
                if element in ['INPUT', 'OUTPUT']:
                    continue
                self.next[self.circuit[element]['OUT']] = basics[element.split('_')[0]](self.start[self.circuit[element]['IN']])
            # print(self.next[1:])
        print('Output: ', ''.join([str(a) for a in self.next[self.circuit['OUTPUT']][:-1]]))
        return self.next[self.circuit['OUTPUT'][:-1]]
    
    def _parse_faults(self, file):
        with open(file) as f:
            lines = [line.rstrip() for line in f]
        faults = [line.split(' ') for line in lines[1:]]
        return np.asarray(faults, dtype=int)[:, 0].tolist(), np.asarray(faults, dtype=int)[:, 1].tolist()
    
    def simulate_faults(self, input_vector, file=None, fault_nodes=None, fault_values=None, verbose=True):
        
        if file is not None:
            fault_nodes, fault_values = self._parse_faults(file)
        
        if fault_nodes is None and file is None:
            fault_nodes = list(range(self.length))

        if fault_values is None and file is None:
            fault_values = [0, 1]*len(fault_nodes)
            fault_nodes = fault_nodes*2
            fault_nodes.sort()
        
        self.fault_values = fault_values
        self.fault_nodes = fault_nodes
        
        # initialize N copies of the circuit with all 0s
        # each row is N copies of that node
        self.start = np.zeros((self.length, len(fault_values)), dtype=int)       
        self.next = np.copy(self.start)
                
        # add the input values
        for i, index in zip(input_vector[:], self.circuit['INPUT']):
            self.next[index, :] = int(i)
        # print(self.next[1:])
                
        # add the faults into the initial state
        for i, (node, value) in enumerate(zip(fault_nodes, fault_values)):
            self.next[node, i] = value
        
        # while not in a steady state, continue looping over the gates in the circuit
        while not np.array_equal(self.start, self.next):
            self.start = np.copy(self.next)
            for element in self.circuit.keys():
                if element in ['INPUT', 'OUTPUT']:
                    continue
                self.next[self.circuit[element]['OUT']] = basics[element.split('_')[0]](self.start[self.circuit[element]['IN']])
                # reset faults
                for i, (node, value) in enumerate(zip(fault_nodes, fault_values)):
                    self.next[node, i] = value
        outputs = [''.join([str(int(a)) for a in self.next[self.circuit['OUTPUT'], n][:-1]]) for n in range(len(fault_nodes))]
        output = ''
        with open(f'fault_analysis_{self.file.split("/")[1]}', 'a') as f:
            f.write(f'Circuit: {self.file}\nInput Vector: {input_vector}\nFault-Free Output: {outputs[-1]}\n')
            count = 0
            for z, output in enumerate(outputs[:-1]):
                if output != outputs[-1]:
                    count += 1
            if file:
                f.write(f'FAULTS DETECTED: {count} ({int(100*count/(len(fault_nodes)))}%)\n')
            elif not file:
                f.write(f'FAULTS DETECTED: {count} ({int(100*count/(len(fault_nodes)-4))}%)\n')
            for z, output in enumerate(outputs[:-1]):
                if output != outputs[-1]:
                    f.write(f'    {int(fault_nodes[z])} stuck at {int(fault_values[z])}\n')
            f.write('\n')
        with open(f'fault_analysis_{self.file.split("/")[1]}') as file:
            f = file.read()
        ff = f.split('\n\n')
        for x in ff[-2].splitlines():
            print(x)
        return None

## S27

In [2]:
file = 'Circuit Inputs/s27.txt'
c = Circuit(file)

In [3]:
test='0000010'
c.simulate_faults(test, file=None)

Circuit: Circuit Inputs/s27.txt
Input Vector: 0000010
Fault-Free Output: 0100
FAULTS DETECTED: 16 (40%)
    1 stuck at 1
    2 stuck at 1
    5 stuck at 1
    6 stuck at 1
    7 stuck at 1
    8 stuck at 0
    9 stuck at 0
    10 stuck at 1
    11 stuck at 1
    14 stuck at 1
    15 stuck at 0
    16 stuck at 0
    17 stuck at 0
    18 stuck at 0
    19 stuck at 0
    20 stuck at 0


## S298f_2

In [4]:
file = 'Circuit Inputs/s298f_2.txt'
c = Circuit(file)

In [ ]:
test = '01010000000000000'
c.simulate_faults(test)

## S344f_2

In [6]:
file = 'Circuit Inputs/s344f_2.txt'
c = Circuit(file)

In [ ]:
test = '001000000000000000000000'
c.simulate_faults(test)

## S349f_2

In [8]:
file = 'Circuit Inputs/s349f_2.txt'
c = Circuit(file)

In [ ]:
tests = '000000000000000000000000'
c.simulate_faults(test)

## S344ff_2

In [10]:
file = 'Circuit Inputs/s344ff_2.txt'
c = Circuit(file)

In [ ]:
test = '001000000000000000000000'
c.simulate_faults(test)